<a href="https://colab.research.google.com/github/madhusudhanrao-ppm/devrel-colab/blob/main/ListOCIBucketsinCompartment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "oci[adk]" oci

In [ ]:
import os
oci_dir = '/root/.oci'
if not os.path.exists(oci_dir):
    os.makedirs(oci_dir)

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Copy OCI config file to Colab environment
!cp /content/drive/MyDrive/OCI_KEY/config ~/.oci/config
# Copy private key file to Colab environment
!cp /content/drive/MyDrive/OCI_KEY/oradbclouducm_sso.pem $oci_dir/oradbclouducm_sso.pem

In [ ]:
import oci
from oci.object_storage import ObjectStorageClient
from oci.identity import IdentityClient

def list_buckets_in_compartment(compartment_id, region='us-phoenix-1'):
    """
    List all Object Storage buckets in a specified compartment.

    Args:
        compartment_id (str): The OCID of the compartment
        region (str): The OCI region (default: us-ashburn-1)

    Returns:
        list: List of bucket objects
    """
    try:
        # Initialize the OCI config (reads from ~/.oci/config by default)
        config = oci.config.from_file()

        # Set the region if different from default
        config['region'] = region

        # Initialize Object Storage client
        object_storage_client = ObjectStorageClient(config)

        # Get the namespace (required for listing buckets)
        namespace = object_storage_client.get_namespace().data
        print(f"Namespace: {namespace}")
        print(f"Compartment ID: {compartment_id}")
        print("-" * 60)

        # List all buckets in the compartment
        buckets_response = object_storage_client.list_buckets(
            namespace_name=namespace,
            compartment_id=compartment_id
        )

        buckets = buckets_response.data

        if not buckets:
            print("No buckets found in this compartment.")
            return []

        print(f"Found {len(buckets)} bucket(s):\n")

        for bucket in buckets:
            print(f"  Bucket Name: {bucket.name}")
            #print(f"  Created: {bucket.time_created}")
            #print(f"  Compartment ID: {bucket.compartment_id}")
            #print(f"  Storage Tier: {bucket.storage_tier}")
            print()

        return buckets

    except oci.exceptions.ServiceError as e:
        print(f"OCI Service Error: {e}")
        raise
    except Exception as e:
        print(f"Error: {e}")
        raise


def list_buckets_with_authentication(
    config_file_path=None,
    profile_name='DEFAULT',
    compartment_id=None,
    region='us-phoenix-1'
):
    """
    List buckets with explicit authentication configuration.

    Args:
        config_file_path (str): Path to OCI config file (default: ~/.oci/config)
        profile_name (str): Profile name in config file (default: DEFAULT)
        compartment_id (str): The OCID of the compartment
        region (str): The OCI region
    """
    try:
        # Load config from file
        if config_file_path:
            config = oci.config.from_file(config_file_path, profile_name)
        else:
            config = oci.config.from_file(profile_name=profile_name)

        config['region'] = region

        # Validate the config
        oci.config.validate_config(config)

        # Initialize clients
        identity_client = IdentityClient(config)
        object_storage_client = ObjectStorageClient(config)

        # Get the namespace
        namespace = 'oradbclouducm'  # Replace with your actual namespace if known
        print(f"Namespace: {namespace}")

        # Get compartment details (optional, for verification)
        if compartment_id:
            try:
                compartment = identity_client.get_compartment(compartment_id).data
                print(f"Compartment: {compartment.name}")
            except Exception as e:
                print(f"Could not fetch compartment details: {e}")

        # List buckets
        print(f"Listing buckets in namespace: {namespace}")
        print("-" * 60)

        buckets_response = object_storage_client.list_buckets(
            namespace_name=namespace,
            compartment_id=compartment_id
        )

        buckets = buckets_response.data

        if buckets:
            print(f"\nFound {len(buckets)} bucket(s):\n")
            for bucket in buckets:
                print(f"  • {bucket.name}")
                print(f"    - Created: {bucket.time_created}")
                print(f"    - Tier: {bucket.storage_tier}")
                print()
        else:
            print("No buckets found.")

        return buckets

    except oci.exceptions.ConfigFileNotFound as e:
        print(f"Config file not found: {e}")
        raise
    except Exception as e:
        print(f"Error: {e}")
        raise


if __name__ == "__main__":
    # Example usage
    # Replace these values with your actual OCI compartment ID and region

    COMPARTMENT_ID = "ocid1.compartment.oc1..aaaaaaaaud6tkdn6n23cbvc4hexs6n4hggetkwo4viqyneyroixcmj54u32q"
    REGION = "us-phoenix-1"  # Change to your region

    print("=== OCI Object Storage - List Buckets ===\n")

    try:
        # Method 1: Using default config from ~/.oci/config
        print("Method 1: Using default OCI config\n")
        list_buckets_in_compartment(COMPARTMENT_ID, REGION)

    except Exception as e:
        print(f"Method 1 failed. Make sure ~/.oci/config exists and contains valid credentials.\n")

        # Method 2: Using explicit config file path and profile
        print("Method 2: Using explicit config file path\n")
        try:
            config_path = "~/.oci/config"  # Adjust this path
            list_buckets_with_authentication(
                config_file_path=config_path,
                profile_name='DEFAULT',
                compartment_id='ocid1.compartment.oc1..aaaaaaaaud6tkdn6n23cbvc4hexs6n4hggetkwo4viqyneyroixcmj54u32q',
                region=REGION
            )
        except Exception as e2:
            print(f"Method 2 also failed: {e2}")




